In [1]:
import pandas as pd
import pyodbc
from connector import get_connection

In [2]:
Matcho_concepts = pd.read_csv("data\macho_concept_mapped.csv")

In [3]:
matcho_concept_id = tuple(Matcho_concepts['concept_id'])

In [4]:
conn = get_connection()

# Load data from cdm.SQL Server with SQL queries for initial filtering
def load_data():
    # getting the person_id of the persons with the concepts

    print("Getting the person_id of the persons with the concepts")
    observation = pd.read_sql(f"SELECT * FROM cdm.observation WHERE observation_concept_id IN {matcho_concept_id}", conn)
    measurement = pd.read_sql(f"SELECT * FROM cdm.measurement WHERE measurement_concept_id IN {matcho_concept_id}", conn)
    procedure_occurrence = pd.read_sql(f"SELECT * FROM cdm.procedure_occurrence WHERE procedure_concept_id IN {matcho_concept_id}", conn)
    condition_occurrence = pd.read_sql(f"SELECT * FROM cdm.condition_occurrence WHERE condition_concept_id IN {matcho_concept_id}", conn)
    drug_exposure = pd.read_sql(f"SELECT * FROM cdm.drug_exposure WHERE drug_exposure_id IN {matcho_concept_id}", conn)

    # join all together
    person_with_concepts_df= pd.concat([observation, measurement, procedure_occurrence, condition_occurrence, drug_exposure])

    # drop duplicates
    person_with_concepts_df = person_with_concepts_df.drop_duplicates(subset=['person_id'])

    return person_with_concepts_df, observation, measurement, procedure_occurrence, condition_occurrence, drug_exposure

def persons_with_concepts(filter_to_persons_with_concepts):

    person = pd.read_sql("SELECT person_id, gender_concept_id, year_of_birth, month_of_birth, day_of_birth FROM cdm.person", conn)
    df = pd.merge(person, filter_to_persons_with_concepts, on='person_id')
    df['day'] = df['day_of_birth'].fillna(1)
    df['date_of_birth'] = pd.to_datetime(df[['year_of_birth', 'month_of_birth', 'day']].astype(int).astype(str).agg('-'.join, axis=1))
    df = df[['person_id', 'gender_concept_id', 'date_of_birth']]
    return df

def get_all_concepts(persons_with_concepts, observation, measurement, procedure_occurrence, condition_occurrence, drug_exposure):
    person_df = persons_with_concepts

    def process_domain(df, person_df, concept_id_col, date_col, domain_name):
        df = df[['person_id', concept_id_col, date_col]]
        df = pd.merge(df, person_df, on='person_id')
        df = df.rename(columns={concept_id_col: 'concept_id', date_col: 'visit_date'})
        df['domain'] = domain_name
        return df

    observation_df = process_domain(observation, person_df, 'observation_concept_id', 'observation_date', 'observation')
    measurement_df = process_domain(measurement, person_df, 'measurement_concept_id', 'measurement_date', 'measurement')
    procedure_df = process_domain(procedure_occurrence, person_df, 'procedure_concept_id', 'procedure_date', 'procedure')
    condition_df = process_domain(condition_occurrence, person_df, 'condition_concept_id', 'condition_start_date', 'condition')
    drug_df = process_domain(drug_exposure, person_df, 'drug_concept_id', 'drug_exposure_start_date', 'drug')

    union_df = pd.concat([measurement_df, procedure_df, observation_df, condition_df, drug_df]).drop_duplicates(subset=['person_id', 'concept_id'])

    union_df['visit_date'] = pd.to_datetime(union_df['visit_date'])
    union_df['date_diff'] = (union_df['visit_date'] - union_df['date_of_birth']).dt.days
    union_df['age'] = union_df['date_diff'] / 365

    return union_df

def filter_to_women_of_reproductive_age(get_all_concepts):
    df = get_all_concepts
    df = df[(df['gender_concept_id'] == 8532) & (df['age'] >= 15) & (df['age'] < 56)]
    return df

def persons_without_concepts(filter_to_persons_with_concepts):

    person = pd.read_sql("SELECT * FROM cdm.person", conn)

    preg_df = filter_to_persons_with_concepts[['person_id']].copy()
    preg_df['has_concept'] = 1
    df = pd.merge(person, preg_df, on='person_id', how='left')
    df = df[df['has_concept'].isnull()]
    df['day'] = df['day_of_birth'].fillna(1)
    df['month_of_birth'] = df['month_of_birth'].fillna(1)
    df['year_of_birth'] = df['year_of_birth'].fillna(df['year_of_birth'].mean())

    df['date_of_birth'] = df[['year_of_birth', 'month_of_birth', 'day']].astype(int).astype(str).agg('-'.join, axis=1)
    # df['date_of_birth'] = pd.to_datetime(df[['year_of_birth', 'month_of_birth', 'day']].astype(int).astype(str).agg('-'.join, axis=1))
    df = df[['person_id', 'gender_concept_id', 'date_of_birth']]
    return df

def get_count_and_proportion(filter_to_women_of_reproductive_age):
    df = filter_to_women_of_reproductive_age

    total = df['person_id'].nunique()
    print(total)

    count_df = df.groupby(['concept_id'])['person_id'].nunique().reset_index(name='count')
    count_df['proportion'] = count_df['count'] / total

    count_df = count_df[(count_df['concept_id'] != 0)]

    return count_df

def filter_to_frequent_concepts(get_count_and_proportion):
    df = get_count_and_proportion
    df = df[df['count'] > 1000]
    return df

def get_frequent_concepts(filter_to_frequent_concepts, persons_without_concepts, observation, measurement, procedure_occurrence, condition_occurrence, drug_exposure):
    concept_df = filter_to_frequent_concepts[['concept_id']]
    person_df = persons_without_concepts

    def process_domain(df, person_df, concept_id_col, date_col):
        df = df[['person_id', concept_id_col, date_col]]
        df = pd.merge(df, person_df, on='person_id')
        df = df.rename(columns={concept_id_col: 'concept_id', date_col: 'visit_date'})
        return df

    observation_df = process_domain(observation, person_df, 'observation_concept_id', 'observation_date')
    measurement_df = process_domain(measurement, person_df, 'measurement_concept_id', 'measurement_date')
    procedure_df = process_domain(procedure_occurrence, person_df, 'procedure_concept_id', 'procedure_date')
    condition_df = process_domain(condition_occurrence, person_df, 'condition_concept_id', 'condition_start_date')
    drug_df = process_domain(drug_exposure, person_df, 'drug_concept_id', 'drug_exposure_start_date')

    union_df = pd.concat([measurement_df, procedure_df, observation_df, condition_df, drug_df]).drop_duplicates(subset=['person_id', 'concept_id'])

    union_df['visit_date'] = pd.to_datetime(union_df['visit_date'])
    union_df['date_of_birth'] = pd.to_datetime(union_df['date_of_birth'])

    union_df['date_diff'] = (union_df['visit_date'] - union_df['date_of_birth']).dt.days
    union_df['age'] = union_df['date_diff'] / 365

    return union_df

def get_count_and_proportion_for_controls(get_frequent_concepts):
    df = get_frequent_concepts

    total = df['person_id'].nunique()
    print(total)

    count_df = df.groupby(['concept_id'])['person_id'].nunique().reset_index(name='count_controls')

    return count_df

def count_and_proportion_all(get_count_and_proportion_for_controls, filter_to_frequent_concepts, get_frequent_concepts):
    cases = filter_to_frequent_concepts
    controls = get_count_and_proportion_for_controls

    df = pd.merge(cases, controls, on='concept_id', how='left')
    df['count_controls'] = df['count_controls'].fillna(1)

    control_total = get_frequent_concepts['person_id'].nunique()

    df['proportion_controls'] = df['count_controls'] / control_total
    df['ratio'] = df['proportion'] / df['proportion_controls']

    return df

def highly_specific_table(count_and_proportion_all):
    df = count_and_proportion_all[count_and_proportion_all['ratio'] > 10]
    return df

def main():
    filter_to_persons_with_concepts_df, person, observation, measurement, procedure_occurrence, condition_occurrence, drug_exposure = load_data()
    persons_with_concepts_df = persons_with_concepts(person, filter_to_persons_with_concepts_df)
    get_all_concepts_df = get_all_concepts(persons_with_concepts_df, observation, measurement, procedure_occurrence, condition_occurrence, drug_exposure)
    filter_to_women_of_reproductive_age_df = filter_to_women_of_reproductive_age(get_all_concepts_df)
    persons_without_concepts_df = persons_without_concepts(person, filter_to_persons_with_concepts_df)
    get_count_and_proportion_df = get_count_and_proportion(filter_to_women_of_reproductive_age_df)
    filter_to_frequent_concepts_df = filter_to_frequent_concepts(get_count_and_proportion_df)
    get_frequent_concepts_df = get_frequent_concepts(filter_to_frequent_concepts_df, persons_without_concepts_df, observation, measurement, procedure_occurrence, condition_occurrence, drug_exposure)
    get_count_and_proportion_for_controls_df = get_count_and_proportion_for_controls(get_frequent_concepts_df)
    count_and_proportion_all_df = count_and_proportion_all(get_count_and_proportion_for_controls_df, filter_to_frequent_concepts_df, get_frequent_concepts_df)
    highly_specific_table_df = highly_specific_table(count_and_proportion_all_df)

OperationalError: (pyodbc.OperationalError) ('08001', '[08001] [Microsoft][ODBC Driver 17 for SQL Server]Named Pipes Provider: Could not open a connection to SQL Server [53].  (53) (SQLDriverConnect); [08001] [Microsoft][ODBC Driver 17 for SQL Server]Login timeout expired (0); [08001] [Microsoft][ODBC Driver 17 for SQL Server]A network-related or instance-specific error has occurred while establishing a connection to SQL Server. Server is not found or not accessible. Check if instance name is correct and if SQL Server is configured to allow remote connections. For more information see SQL Server Books Online. (53)')
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
person_df, observation, measurement, procedure_occurrence, condition_occurrence, drug_exposure = load_data()

Getting the person_id of the persons with the concepts


C:\Users\tiger\AppData\Local\Temp\ipykernel_26844\3572867221.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  person_with_concepts_df= pd.concat([observation, measurement, procedure_occurrence, condition_occurrence, drug_exposure])


In [ ]:
persons_with_concepts_df = persons_with_concepts(person_df)

In [ ]:
get_all_concepts_df = get_all_concepts(persons_with_concepts_df, observation, measurement, procedure_occurrence, condition_occurrence, drug_exposure)

In [ ]:
filter_to_women_of_reproductive_age_df = filter_to_women_of_reproductive_age(get_all_concepts_df)

In [ ]:
persons_without_concepts_df = persons_without_concepts(person_df)

In [ ]:
get_count_and_proportion_df = get_count_and_proportion(filter_to_women_of_reproductive_age_df)

254858


In [ ]:
filter_to_frequent_concepts_df = filter_to_frequent_concepts(get_count_and_proportion_df)

In [ ]:
get_frequent_concepts_df = get_frequent_concepts(filter_to_frequent_concepts_df, persons_without_concepts_df, observation, measurement, procedure_occurrence, condition_occurrence, drug_exposure)


In [83]:

get_count_and_proportion_for_controls_df = get_count_and_proportion_for_controls(get_frequent_concepts_df)
# count_and_proportion_all_df = count_and_proportion_all(get_count_and_proportion_for_controls_df, filter_to_frequent_concepts_df, get_frequent_concepts_df)
# highly_specific_table_df = highly_specific_table(count_and_proportion_all_df)

0


In [87]:
get_count_and_proportion_for_controls_df

,concept_id,count_controls


In [86]:
count_and_proportion_all_df

,concept_id,count,proportion,count_controls,proportion_controls,ratio
0,74698,7254,0.028463,1.0,inf,0.0
1,75605,4783,0.018767,1.0,inf,0.0
2,76482,4268,0.016747,1.0,inf,0.0
3,197625,1132,0.004442,1.0,inf,0.0
4,199076,2649,0.010394,1.0,inf,0.0
5,434714,2054,0.008059,1.0,inf,0.0
6,436176,3260,0.012791,1.0,inf,0.0
7,436747,2044,0.008020,1.0,inf,0.0
8,437611,1404,0.005509,1.0,inf,0.0
9,438491,1818,0.007133,1.0,inf,0.0


In [85]:
count_and_proportion_all_df

,concept_id,count,proportion,count_controls,proportion_controls,ratio
0,74698,7254,0.028463,1.0,inf,0.0
1,75605,4783,0.018767,1.0,inf,0.0
2,76482,4268,0.016747,1.0,inf,0.0
3,197625,1132,0.004442,1.0,inf,0.0
4,199076,2649,0.010394,1.0,inf,0.0
5,434714,2054,0.008059,1.0,inf,0.0
6,436176,3260,0.012791,1.0,inf,0.0
7,436747,2044,0.008020,1.0,inf,0.0
8,437611,1404,0.005509,1.0,inf,0.0
9,438491,1818,0.007133,1.0,inf,0.0


In [84]:
highly_specific_table_df

,concept_id,count,proportion,count_controls,proportion_controls,ratio
